In [1]:
import numpy as np
import os, re
from Data import dataExp
%matplotlib inline
from matplotlib import pyplot as plt
import warnings
from sklearn import metrics
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

2023-05-12 13:04:32.175261: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 13:04:33.044554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-12 13:04:34.360282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-12 13:04:34.446666: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [8]:
author = "Caesar"
CI = dataExp.CorpusInterface(corpus_name="text_corpus.pickle", shouldTokenize = False)

Found the existing corpus
abbofloracensis had 1 pieces of work with a total of 34398 characters of text
abelard had 1 pieces of work with a total of 85089 characters of text
acticussincerius had 3 pieces of work with a total of 23788 characters of text
addison had 1 pieces of work with a total of 1764 characters of text
adso had 1 pieces of work with a total of 13551 characters of text
aelredus had 1 pieces of work with a total of 118173 characters of text
agnes had 1 pieces of work with a total of 74784 characters of text
alanus had 1 pieces of work with a total of 195061 characters of text
albericodamarcellise had 1 pieces of work with a total of 172 characters of text
albertanus had 1 pieces of work with a total of 18033 characters of text
albertofaix had 1 pieces of work with a total of 85671 characters of text
alcuin had 1 pieces of work with a total of 2642 characters of text
aleandrogerolamo had 2 pieces of work with a total of 13499 characters of text
alfonsi had 1 pieces of wo

In [9]:
text = CI.get_text_for_author(author=author,shouldShuffle=True).replace("\t","")
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"The character vocabulary is : {chars}")

# We just create a mapping between our character vocabulary
# and their corresponding integer value, and define lambda funcs to do this mapping for us

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

batch_size = 64 
context_size = 256
max_iters = 5000 
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = .2

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(context_size, context_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
            This performs the self-attention that we worked through did in the notebook
        """
        batch, time, channel = x.shape

        k = self.key(x) # (batch, time, head_size)
        q = self.query(x) # (batch, time, head_size)

        # compute the affinities/scaled attention scores
        weights = q @ k.transpose(-2, -1) * channel **-.5
        # don't want to interact with subsequent time step tokens
        # i.e. makes it a decoder block
        weights = weights.masked_fill(self.tril[:time, :time] == 0, float('-inf'))
        # make the probability nicely distributed
        weights = F.softmax(weights, dim=-1) # (batch, time, time)
        weights = self.dropout(weights)
        # weighted aggreagation of values
        v = self.value(x)
        out = weights @ v 
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ just a simple linear layer followed by non-linearity, as there 
        was a FF part in the paper too

        on a per-token layer
    """ 
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
        # last nn.Linear is the projectin layer back into residual pathway

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication intersperesed with calculation """

    def __init__(self, n_embd, n_head):
        """ n_embd: embedding dimension, n_head: the number of heads we want """
        super().__init__()
        head_size = n_embd//n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd= FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # make them residual connnections by doing x + 
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(context_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_final = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        batch, time = idx.shape

        # idx and targets are both (batch, time) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (batch, time, channel)
        pos_emb = self.position_embedding_table(torch.arange(time, device=device)) # (time, channel)
        x = tok_emb + pos_emb # (batch,time, channel)
        x = self.blocks(x) # (batch,time, channel)
        x = self.ln_final(x) # (batch,time, channel)
        logits = self.lm_head(x) # (batch,time,vocab_size)

        if targets is None:
            loss = None
        else:
            batch, time, channel = logits.shape
            logits = logits.view(batch*time, channel)
            targets = targets.view(batch*time)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (batch,time) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last context_size tokens
            idx_cond = idx[:, -context_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (batch, channel)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (batch, channel)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (batch, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (batch, time step+1)
        return idx

The character vocabulary is : [' ', '!', '"', "'", '(', ')', '+', ',', '-', '.', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z']


In [5]:
print(author, len(text))
model = LanguageModel()
model.to(device)
model.load_state_dict(torch.load(os.getcwd()+f"/LatinTransformer/{author}_model.pt"))

Caesar 616449


<All keys matched successfully>

In [10]:
prompt = "Olim Rufo nomine iuvenis ingeniosus erat,".lower()
context_values = encode(prompt)
context = torch.tensor(context_values, dtype=torch.long, device=device).reshape((len(context_values),1))

gen =model.generate(context, max_new_tokens=220)[0].tolist()
print(prompt+ decode(gen))

olim rufo nomine iuvenis ingeniosus erat,od minores arbitrium esse ipsi; quibus in contuberioribus rei erant, ut eam civitas quae ad sibi bellum agebant, quod pro cum flumine hostibus aliter reverterant, ne non solum consulebant atque ibi praesidia. cum venisset
